In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ---------------------------------------------------
# 1) shape_features 함수:
#    - 신호의 평균 0, 표준편차 1로 스케일링(z-normalization)
#    - 선형보간(Interpolation)을 통해 고정 길이로 변환
#    - -> "순수 형태"를 나타내는 벡터를 획득
# ---------------------------------------------------
def shape_features(signal, output_length=100):
    """
    signal        : 1D numpy array
    output_length : 보간 후 만들 벡터의 길이 (예: 100)

    return        : shape_vec (크기가 output_length인 벡터)
    """

    # (1) 오프셋 제거(평균 0)
    signal = signal - np.mean(signal)

    # (2) 크기(표준편차) 제거(분산 1)
    std_val = np.std(signal)
    if std_val < 1e-12:
        # 신호가 상수에 가깝다면, 그냥 0으로만 이루어진 벡터
        signal = np.zeros_like(signal)
    else:
        signal = signal / std_val

    # (3) 길이 N -> output_length로 선형 보간
    N = len(signal)
    if N <= 1:
        # 혹시 길이가 1 이하라면, 그냥 output_length만큼 0으로 만듦
        return np.zeros(output_length, dtype=float)

    old_x = np.arange(N)
    new_x = np.linspace(0, N - 1, output_length)  # 0 ~ (N-1)을 output_length로 균등 분할
    f = interp1d(old_x, signal, kind='linear')
    shape_vec = f(new_x)  # shape_vec.shape = (output_length,)

    return shape_vec


# ---------------------------------------------------
# 2) CSV 파일 경로 지정
# ---------------------------------------------------
file_path_real = "/content/MyDrive/2025CAP/CSV/real/*.csv"  # 진짜 서명 CSV 경로
file_path_fake = "/content/MyDrive/2025CAP/CSV/fake/*.csv"  # 위조 서명 CSV 경로

# ---------------------------------------------------
# 3) 데이터 로드 + shape-based feature 구성
# ---------------------------------------------------
X_data = []
y_data = []

# (a) 진짜 서명
real_files = glob.glob(file_path_real)
for fpath in real_files:
    df = pd.read_csv(fpath, skiprows=1)  # 필요 시 skiprows 조정
    df.dropna(inplace=True)

    # 마지막 열을 센서 신호로 가정
    pressure_column = df.columns[-1]
    pressure_array = df[pressure_column].values

    shape_vec = shape_features(pressure_array, output_length=100)
    X_data.append(shape_vec)
    y_data.append(1)  # 진짜

# (b) 위조 서명
fake_files = glob.glob(file_path_fake)
for fpath in fake_files:
    df = pd.read_csv(fpath, skiprows=1)
    df.dropna(inplace=True)

    pressure_column = df.columns[-1]
    pressure_array = df[pressure_column].values

    shape_vec = shape_features(pressure_array, output_length=100)
    X_data.append(shape_vec)
    y_data.append(0)  # 위조

X_data = np.array(X_data)
y_data = np.array(y_data)

print("📊 총 샘플 수 =", len(X_data))
print("📊 X_data shape =", X_data.shape, "(샘플 수, 100)")
print("📊 y_data shape =", y_data.shape)

# ---------------------------------------------------
# 4) 분류 (SVM 예시)
# ---------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, random_state=42, stratify=y_data
)

model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("🎯 정확도(Accuracy) =", acc)
print("📊 혼동행렬(Confusion Matrix)\n", cm)
print("📄 상세 보고서\n", classification_report(y_test, y_pred, digits=4))

# ---------------------------------------------------
# (선택) 시각화 - 첫 번째 몇 개 샘플의 shape_vec 비교
# ---------------------------------------------------
plt.figure(figsize=(10, 4))
for i in range(min(10, len(X_data))):
    plt.plot(X_data[i], label=f"Sample {i} (y={y_data[i]})")
plt.title("Shape-based vectors (z-normalized + interpolation)")
plt.xlabel("Normalized Time Index (0~99)")
plt.ylabel("Amplitude (Z-normalized)")
plt.legend()
plt.grid(True)
plt.show()


📊 총 샘플 수 = 0
📊 X_data shape = (0,) (샘플 수, 100)
📊 y_data shape = (0,)


ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

MessageError: Error: credential propagation was unsuccessful